# CNN

## Red entrenada

Importando bibliotecas

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms

Inicializando parámetros para el uso de la GPU

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
train_kwargs = {'batch_size': 500}
test_kwargs = {'batch_size': 500}
if use_cuda:
    cuda_kwargs = {'num_workers': 1, 'pin_memory': True, 'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

Descargando conjunto de datos (aplica transformaciones)

In [3]:
transform = transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.5,), (0.3,)) ])

dataset1 = datasets.MNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.MNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 15688213.55it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 507734.04it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4359682.89it/s]


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 3037877.34it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



## Red convolucional

Definición del modelo

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch,
                batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss,
        correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))

Main execution

In [5]:
model = Net().to(device)
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(5):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.311967
Train Epoch: 0 [10000/60000 (17%)]	Loss: 0.491292
Train Epoch: 0 [20000/60000 (33%)]	Loss: 0.280689
Train Epoch: 0 [30000/60000 (50%)]	Loss: 0.223724
Train Epoch: 0 [40000/60000 (67%)]	Loss: 0.208582
Train Epoch: 0 [50000/60000 (83%)]	Loss: 0.160930


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()



Test set: Average loss: 0.0703, Accuracy: 9773/10000 (97.73%)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.103978
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.125467
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.143290
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.083240
Train Epoch: 1 [40000/60000 (67%)]	Loss: 0.065785
Train Epoch: 1 [50000/60000 (83%)]	Loss: 0.141876

Test set: Average loss: 0.0435, Accuracy: 9861/10000 (98.61%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.086841
Train Epoch: 2 [10000/60000 (17%)]	Loss: 0.076105
Train Epoch: 2 [20000/60000 (33%)]	Loss: 0.076107
Train Epoch: 2 [30000/60000 (50%)]	Loss: 0.105688
Train Epoch: 2 [40000/60000 (67%)]	Loss: 0.091127
Train Epoch: 2 [50000/60000 (83%)]	Loss: 0.093661

Test set: Average loss: 0.0388, Accuracy: 9862/10000 (98.62%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.084058
Train Epoch: 3 [10000/60000 (17%)]	Loss: 0.069267
Train Epoch: 3 [20000/60000 (33%)]	Loss: 0.070628
Train Epoch: 3 [30000/60000 (50%)]	Loss: 0.059437
Train Epoch: 3 [40000/6

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.046530
Train Epoch: 4 [10000/60000 (17%)]	Loss: 0.058467
Train Epoch: 4 [20000/60000 (33%)]	Loss: 0.041516
Train Epoch: 4 [30000/60000 (50%)]	Loss: 0.047902
Train Epoch: 4 [40000/60000 (67%)]	Loss: 0.043068
Train Epoch: 4 [50000/60000 (83%)]	Loss: 0.020944

Test set: Average loss: 0.0321, Accuracy: 9889/10000 (98.89%)

## Tarea 1

Entrenar la red convolucional pero esta vez con el conjunto de datos Fashion-MNIST

In [ ]:
transform = transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.5,), (0.3,)) ])

dataset1 = datasets.FashionMNIST('../data', train=True, download=True, transform=transform)
dataset2 = datasets.FashionMNIST('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

In [ ]:
# TODO


Reportar el desempeño alcanzado con la red convolucional en el conjunto de datos Fashion-MNIST

## Tarea 2

Entrenar la red convolucional pero esta vez con el conjunto de datos CIFAR10

In [ ]:
transform = transforms.Compose([ transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5,), (0.3,0.3,0.3,)) ])

dataset1 = datasets.CIFAR10('../data', train=True, download=True, transform=transform)
dataset2 = datasets.CIFAR10('../data', train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, **train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

from torchvision.utils import make_grid
import matplotlib.pyplot as plt
for images, labels in train_loader:
    fig, ax = plt.subplots(figsize=(10, 10))
    ax.set_xticks([]); ax.set_yticks([])
    ax.imshow(make_grid(images[:64], nrow=8).permute(1, 2, 0))
    break

In [ ]:
# TODO


Reportar el desempeño alcanzado con la red convolucional en el conjunto de datos CIFAR10